In [4]:
import sys
import time
from pathlib import Path

sys.path.append(str(Path('../').resolve()))
assert 'python-nostr' in [Path(path).name for path in sys.path]

from nostr.key import PrivateKey, PublicKey
from client import ConnectedClient

## make a helper method to get a private key from disk
or allow user input

In [5]:
def get_private_key() -> str:
    key_file = Path('./private_key.txt') # saving a private key for testing

    if not key_file.exists():
        pk_hex = input()
        if len(pk_hex.strip()) == 0:
            pk_hex = PrivateKey().hex()
            print(f'generated new private key: {pk_hex}')
        else:
            print('using user private key')
    else:
        with open(key_file, 'r') as f:
            pk_hex = f.read()
            # private_key = PrivateKey(bytes.fromhex(hex))
            private_key = PrivateKey.from_hex(pk_hex)
        public_key = private_key.public_key
        print(f'Loaded private key from {key_file}')
        print(f'Public key: {public_key.bech32()}')
    return pk_hex

# client
instantiating a basic client and testing connections

In [3]:
pk_hex = get_private_key()

with ConnectedClient(ssl_options={'cert_reqs': ssl.CERT_NONE},
                     private_key_hex=pk_hex) as connected_client:
    print(f'loaded public key is {connected_client.public_key.bech32()}\n')
    for relay in connected_client.relay_manager:
        assert relay.ws.sock is not None
        print(f'connection to {relay.url} is alive!')

for relay in connected_client.relay_manager:
    assert relay.ws.sock is None
    print(f'connection to {relay.url} is dead.')

Loaded private key from private_key.txt
Public key: npub19nqc7plddvevd8sgfpahzuput0q9hwrfhk8crj9k73fpg8qwjx3sk6224n


NameError: name 'ssl' is not defined

# make a simple text input client

we will instantiate it with the proper options first

In [6]:
from client import TextInputClient
import ssl

pk_hex = get_private_key()

text_client = TextInputClient(ssl_options={'cert_reqs': ssl.CERT_NONE}, private_key_hex=pk_hex)

Loaded private key from private_key.txt
Public key: npub19nqc7plddvevd8sgfpahzuput0q9hwrfhk8crj9k73fpg8qwjx3sk6224n


## run it
with a simple `with` context. the `__enter__` method calls the `run` command of the client so we can just call `pass` so that nothing else happens after we exit the `run` method before the we `__exit__` the clients and the connections are closed.

In [7]:
with text_client:
    pass

select a command:
                	0	E(x)it
                	1	publish note
                	2	get last 10 notes by you
                	3	get last 10 from hex of author
                	4	delete an event
                	5	check deletions
                
author: 2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3
event id:422c574a74f7668a38704f5b41dec69e3d18c91c3ebe14e1851ae9d54ee8ae02
	needs to be deleted
author: 2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3
event id:0e639e25b666c32f219389f322641011f9f0506c8e0b407bcb574f4d503b9c47
	need to delete
author: 2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3
event id:248c06f4a97b305a61b868f6f7f2813ae26804841325ae98d748ed1f15c75f16
	testing
author: 2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3
event id:d8e2f9537ff58882b3e6ff95b44ad705c3f7f51208d8c17875921094f0cd25a7
	testing deletion
author: 2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3
event id:036f6bbf4

In [9]:
text_client.message_store['33af995f8df811609b91b31469b2df604f693b70664abb3aa572f318a0f9ed5e']

{'id': '33af995f8df811609b91b31469b2df604f693b70664abb3aa572f318a0f9ed5e',
 'pubkey': '2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3',
 'created_at': 1672100307,
 'kind': 1,
 'tags': [],
 'content': 'do something',
 'sig': 'bc5c9496f337df686e7b5044acb53cbce8b083d675ff676faea0e7fa01d8692539c033cf51977e258f35db47ebfb01ffc0055d408501e4e69151ad74186ca6eb'}